In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

#브라우저 꺼짐 방지
chrome_options = Options()
chrome_options.add_experimental_option('detach', True)
keyword = 'gis'
url = f'https://www.dbpia.co.kr/search/topSearch?searchOption=all&query={keyword}'

service = Service('chromedriver.exe')
browser = webdriver.Chrome(service=service, options = chrome_options)
browser.implicitly_wait(10)
browser.maximize_window()
browser.get(url)
#Selenium - Beautifulsoup 연동방법


list_title = []
list_journal = []
list_date = []
list_pages = []


for i in range(2, 12):
    pgnum = browser.find_element(By.CSS_SELECTOR, '#pageList > a:nth-child(' + str(i) + ')')
    pgnum.click()
    time.sleep(2)  # Add a delay to ensure the new page is fully loaded
    html = browser.page_source  # Get the updated page source
    soup = BeautifulSoup(html, 'html.parser')  # Create a new soup object with the updated HTML
    infos = soup.select("article.thesis__summary")

    for info in infos:
        try:
            title = info.select_one('h2.thesis__tit').text
            journal = info.select_one('a.nodeIprd').text
            date = info.select_one('section.thesisAdditionalInfo>span:nth-of-type(1)').text
            pages = info.select_one('section.thesisAdditionalInfo>span:nth-of-type(2)').text
            list_title.append(title)
            list_journal.append(journal)
            list_date.append(date)
            list_pages.append(pages)
        except AttributeError:
            continue



    


In [5]:
import pandas as pd
result = pd.DataFrame({
    '논문명': list_title,
    '학회지': list_journal,
    '게재일': list_date,
    '페이지': list_pages
})

result.head()

,논문명,학회지,게재일,페이지
0,GIS 기반 로컬상권 형성에 따른 점포집적 특성 - 서울시 성수동 지역의 음식점을 ...,한국실내디자인학회,2023.5,342-347(6pages)
1,설치 장소에 따른 GIS 수명분석에 관한 연구,한국조명·전기설비학회,2023.5,31-31(1pages)
2,A Study on GIS Lifetime Management based on De...,한국조명·전기설비학회,2023.5,103-103(1pages)
3,"한국 역사지리정보의 디지털 이행과 분석, 그리고 역사 GIS의 과제",서강대학교 인문과학연구소,2023.4,153-185(33pages)
4,25.8kV 옥내용 노후 GIS에서 추출한 오링의 상태평가,한국조명·전기설비학회,2023.5,99-99(1pages)


In [6]:
result.to_csv('gis.csv', encoding="utf-8-sig", index= False)